In [6]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time
import datetime
import gspread

In [7]:
SPOTIPY_CLIENT_ID='YOUR-CLIENT-ID'
SPOTIPY_CLIENT_SECRET='YOUR-CLIENT-SECRET'
SPOTIPY_REDIRECT_URI='http://127.0.0.1:9090'
SCOPE = "user-top-read"

In [8]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID, 
                                               client_secret=SPOTIPY_CLIENT_SECRET, 
                                               redirect_uri=SPOTIPY_REDIRECT_URI, 
                                               scope=SCOPE))

In [9]:
top_tracks_short = sp.current_user_top_tracks(limit=10, offset=0, time_range="short_term")

In [ ]:
top_tracks_short

In [11]:
top_tracks_short = sp.current_user_top_tracks(limit=20, offset=0, time_range="short_term")

In [12]:
type(top_tracks_short)

dict

In [13]:
def get_track_ids(time_frame):
    track_ids = []
    for song in time_frame['items']:
        track_ids.append(song['id'])
    return track_ids

In [14]:
track_ids = get_track_ids(top_tracks_short)

In [ ]:
track_ids

In [16]:
track_id = '4iJyoBOLtHqaGxP12qzhQI'

In [17]:
def get_track_features(id):
    meta = sp.track(id)
    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    spotify_url = meta['external_urls']['spotify']
    album_cover = meta['album']['images'][0]['url']
    track_info = [name, album, artist, spotify_url, album_cover]
    return track_info

In [ ]:
get_track_features(track_id)

In [19]:
# loop over track ids 
tracks = []
for i in range(len(track_ids)):
    time.sleep(.5)
    track = get_track_features(track_ids[i])
    tracks.append(track)

In [ ]:
tracks

In [ ]:
# create dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'spotify_url', 'album_cover'])
df.head(5)

In [20]:
gc = gspread.service_account(filename='my-spotify-wrapped-test-ad510490b281.json')

In [23]:
sh = gc.open("My Spotify Wrapped")

In [22]:
worksheet = sh.worksheet("short_term")

In [ ]:
val = worksheet.acell('B5').value
val

In [ ]:
# worksheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

In [24]:
# insert into sheet
def insert_to_gsheet(track_ids):
    # loop over track ids 
    tracks = []
    for i in range(len(track_ids)):
        time.sleep(.5)
        track = get_track_features(track_ids[i])
        tracks.append(track)
    # create dataframe
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'spotify_url', 'album_cover'])
    # insert into google sheet
    gc = gspread.service_account(filename='my-spotify-wrapped-test-ad510490b281.json')
    sh = gc.open('My Spotify Wrapped')
    worksheet = sh.worksheet(f'{time_period}')
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    print('Done')

In [25]:
time_ranges = ['short_term', 'medium_term', 'long_term']
for time_period in time_ranges:
    top_tracks = sp.current_user_top_tracks(limit=20, offset=0, time_range=time_period)
    track_ids = get_track_ids(top_tracks)
    insert_to_gsheet(track_ids)

Done
Done
Done
